# 싱글톤 디자인 패턴 

글로벌하게 접근 가능한 단 한개의 객체만을 허용하는 패턴

주로 로딩, 데이터 베이스 관련 작업, 프린터 스풀러 등의 애플리케이션에서 동일한 리소스에 대한 동시 요청의 충돌을 막기위해
한개의 인스턴스만 필요한 경우에 주로 쓰임.

목적

* 클래스에 대한 단일 객체 생성

* 전역 객체 제공

* 공유된 리소스에 대한 동시 접근 제어 

In [ ]:
## 파이썬 싱글톤 패턴 구현 

1. 한개의 Singleton 클래스 인스턴스 생성

2. 이미 생성된 인스턴스가 있다면 재사용.

In [3]:
class Singleton(object):
    def __new__(cls):
        if not hasattr(cls, 'instance'):
            cls.instance = super(Singleton, cls).__new__(cls)
        return cls.instance
    
s = Singleton()

print("Object created", s)

s1 = Singleton()
print("Object created", s1 )

Object created <__main__.Singleton object at 0x0000020E32032F98>
Object created <__main__.Singleton object at 0x0000020E32032F98>


In [ ]:
## 게으른 초기화 

모듈을 임포트할 때 (아직 객체가 필요하지 않은 시점에) 실수로 미리 생성하는 경우가 있음
게으른 초기화는 인스턴스가 꼭 필요할 때 생성함. 사용할 수 있는 리소스가 제한적인 상황일 때 객체가 꼭 필요한 시점에 생성하는 방식.

다음 코드에서 s = Singleton() 부분은 __init__ 함수를 실행하지만 객체는 생성하지않음.
대신 Singleton.getInstance() 부분에서 객체가 생성됨.

In [8]:
class Singleton:
    __instance = None
    def __init__(self):
        if not Singleton.__instance:
            print("__init__ method called..")
        else:
            print("Instance already created:", self.getInstance())
    @classmethod
    def getInstance(cls):
        if not cls.__instance:
            cls.__instance = Singleton()
        return cls.__instance
    
s = Singleton() ## 클래스를 초기화했지만 객체는 생성되지않음.
print("Object created", Singleton.getInstance()) # 객체 생성
s1 = Singleton()  ## 객체는 이미 생성됐음 

__init__ method called..
__init__ method called..
Object created <__main__.Singleton object at 0x0000020E320B4D68>
Instance already created: <__main__.Singleton object at 0x0000020E320B4D68>


## 모듈 싱글톤

파이썬의 임포트 방식 때문에 모든 모듈은 기본적으로 싱글톤이다. 파이썬의 작동 방식은 다음과 같다.

1. 파이썬 모듈이 임포트 됐는지 확인한다.

2. 임포트 됐다면 해당 객체를 반환하고 안 됐다면 임포트하고 인스턴스 화한다.

3. 모듈은 임포트와 동시에 초기화된다. 하지만 같은 모듈을 다시  임포트하면 초기화하지 않는다. 한개의 객체만 유지하고 반환하는 싱글톤 방식이다.


## 모노스테이트 싱글톤 패턴 

The Monostate Singleton Patten 이름 그대로 모든 객체가 같은 상태를 공유하는 패턴

\_\_dict__변수 (파이썬 특수 변수)를 \_\_shared_state 클래스 변수로 지정함.

파이썬은 \_\_dict__변수내에 클래스 내 모든 객체의 상태를 저장함. 

다음 예제에서는 모든 생성된 인스턴스의 상태를 \_\_shared_state로 지정함.

따라서 b와 b1 인스턴스를 따로 생성해도 한 개의 객체만 만드는 싱를톤 패턴과 달리 두개의 객체가 생성됨.

하지만 객체의 상태를 나타내는 b.\_\_dict__ 와 b1.\_\_dict__은 같음.

따라서 b 객체의 x값을 4로 설정하면 모든 객체가 공유하는 \_\_dict__변수 에 복사되 b1의 x값도 1에서 4로 바뀐다. 

In [33]:
class Borg:
    __shared_state = {"1": "2"}
    def __init__(self):
        self.x = 1
        self.__dict__ = self.__shared_state
        pass
    
b = Borg()
b1 = Borg()
b.x = 4

print("Borg Object 'b': ", b) ## b와 b1은 다른 객체이다. 
print("Borg Object 'b1': ", b1)
print("Object State 'b': ", b.__dict__) ## b와 b1은 상태를 공유한다.
print("Object State 'b1': ", b1.__dict__)


Borg Object 'b':  <__main__.Borg object at 0x0000020E320CAAC8>
Borg Object 'b1':  <__main__.Borg object at 0x0000020E320CAA58>
Object State 'b':  {'1': '2', 'x': 4}
Object State 'b1':  {'1': '2', 'x': 4}


In [12]:
class Borg(object):
    _shared_state = {}
    def __new__(cls, *args, **kwarges):
        obj = super(Borg, cls).__new__(cls, *args, **kwargs)
        obj.__dict__ = cls._shared_sate 
        return obj

## 싱글톤과 메타 클래스

메타클래스는 간단하게 설명하면 클래스의 클래스
즉 클래스는 자신의 메타 클래스의 인스턴스. 메타 클래스를 사용하면 이미 정의된 파이썬 클래스를통해 새로운 형식의
클래스를 생성할 수 있다. 예를 들어 MyClass라는 객체가 있다면 MyKls라는 메타 클래스를 생성해
MyClass의 행위를 재정의할 수 있다.

파이썬에서 모든 것은 객체다. a = 5라면 type(a)는 <type 'int'>를 반환한다.
a는 int형 변수라는 뜻이다. 하지만 type(int)는 <type 'type'> 을 반환한다. int의 메타클래스는 type 클래스라는 의미다.

클래스는 메타클래스가 정의한다. class A구문으로 클래스를 생성하면 파이썬은 A = type(name, bases, dict)을 실행한다.

* name: 클래스명

* base: 베이스 클래스

* dict: 속성값 

이미 정의된 메타클래스 (by the name of MetAKls) 가 있다면 파이썬은 A = MetaKls(names, bases, dict)을 실행해 
클래스를 생성한다.

다음은 파이썬 3.5로 메타클래스를 구현한 코드다

In [39]:
class MyInt(type):
    def __call__(cls, *args, **kwds):
        print("***** Here's My int *****", args)
        print("Now do whatever you want with this objects...")
        return type.__call__(cls, *args, **kwds)
    
class int(metaclass = MyInt):
    def __int__(self, x, y):
        self.x = x
        self.y = y
        
i = int(4, 5)

***** Here's My int ***** (4, 5)
Now do whatever you want with this objects...


TypeError: int() takes no arguments

In [ ]:
왜 오류나는지 확인해야함.

\_\_call__ 메소드는 이미 존재하는 클래스의 객체를 생성할 때 호출되는 파이썬의 특수 메소드다.
위처럼 int(4,5)로 int 클래스를 생성하면 MyInt 메타클래스의 \_\_call__가 호출된다. 객체 생성을 메타클래스가 제어한다는 의미다.
놀랍다.

싱글톤 디자인 패턴과 같은 개념이다. 메타클래스가 클래스와 객체 생성을 제어한다면 싱글톤을 생성하는 용도로 사용할 수 있다는 의미다.
(클래스 생성과 인스턴스 화를 제어하기 위해 메타클래스는 \_\_new__와 \_\_init 메소드를 오버라이드한다.)

다음은 메타클래스를 사용해 싱글톤 패턴을 구현한 예제이다.

In [38]:
class MetaSingleton(type):
    _instances = {}
    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super(MetaSingleton,\
                                       cls).__call__(*args, **kwargs)
        return cls._instances[cls]
    

class Logger(metaclass = MetaSingleton):
    pass

logger1 = Logger()
logger2 = Logger()
print(logger1, logger2 )

<__main__.Logger object at 0x0000020E320CA438> <__main__.Logger object at 0x0000020E320CA438>



## 싱글톤 패턴 사용 사례 1 

데이터베이스 기반 애플리케이션에서 싱글톤 패턴을 적용한 사례를 살펴보자. 데이터베이스에서 데이터를 읽고 쓰는 클라우드 서비스를 
예로 들자. 이 클라우드 서비스에는 데이터 베이스에 접근하는 여러 모듈이 있다고 하자. 각 UI(웹 앱)에서 직접 DB 연산을 수행하는 API를 호출한다.

여러 서비스가 한 개의 DB를 공유하는 구조다. 안정된 클라우드 서비스를 설계하려면 다음 사항들을 반드시 명심해야한다.

* 데이터 베이스의 일관성을 보존해야한다. 연산 간의 충돌이 없어야한다.
* 다수의 DB 연산을 처리하려면 메모리와 CPU를 효율적으로 사용해야 한다. 


In [64]:
import sqlite3
class MetaSingleton(type):
    _instances = {}
    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super(MetaSingleton, cls).__call__(*args, **kwargs)
            return cls._instances[cls]
        
class Database(metaclass = MetaSingleton):
    connection = None
    def connect(self):
        if self.connection is None:
            self.connection = sqlite3.connect("db.sqlite3")
            self.cursorobj = self.connection.cursor()
        return self.cursorobj
    
db1 = Database().connect()
db2 = Database().connect()

print("Database Objects DB1", db1)
print("Database Objects DB2", db2)

AttributeError: 'NoneType' object has no attribute 'connect'

In [ ]:
db1는 되는데 db2는 왜 안되는지 확인 ``

In [51]:
위 코드 분석

1. MetaSingleton 이라는 메타 클래스를 생성. 앞서 설명했듯이 __call__ 파이썬 메서드를 사용해 싱글톤 생성.

2. database 클래스는 MetaSingleton 메타 클래스의 도움으로 싱글톤 역할을 함. 즉 단 한개의 database 클래스 객체만 생성됨

3. 웹 앱이 DB 연산을 요청할 때 마다 database 클래스를 생성하지만 내부적으로는 한 개의 객체만 생성됨.
따라서 데이터 베이스의 동기화가 보장됨. 리소스를 적게 사용해 메모리와 CPU의 사용량 최적화 가능..


단일 웹앱 형태가 아니라 여러 웹앱이 같은 DB에 접속하는 상황이라면 각 웹앱이 DB에 접근하는 싱글톤을 생성하기 때문에
싱그롵ㄴ 패턴에 적합하지않음. DB동기화가 어렵고 리소스 사용량이 많은 경우임. 싱글톤 패턴보다 연결 풀링(Connection Pooling)기법을
사용하는 것이 더 효율적임 

## 싱글톤 패턴 사용 사례 2

이제 인프라 상태를 확인하는 서비스(예를 들어 Nagios)를 구현해보자. 우선 HealthCheck 클래스를 싱글톤으로 구현함.
상태를 확인해야 하는 서버의 목록을 만들고 목록에서 제거된 서버의 상태는 확인하지 않아야한다.

다음 코드를 보면 hc1과 hc2는 동일한 객체임.

addServer() 메소드는 서버를 목록에 추가함. 서비스는 목록의 각 서버의 상태를 확인함.
changeServer() 메소드는 서버를 목록에서 제거하고 새로운 서버를 추가함. 따라서 서비스가 두번 째 실행될 떄는 바뀐 서버 목록을 참조함.

위 내용을 싱글톤 패턴으로 구현한것이 아래 코드임.
서버가 추가 또는 제거 되면 각 HealthCheck 객체는 똑같이 바뀐 목록을 참조해야함. 

In [69]:
class HealthCheck:
    _instance = None
    def __new__(cls, *args, **kwargs):
        if not HealthCheck._instance:
            HealthCheck._instance = super(HealthCheck, cls).__new__(cls, *args, **kwargs)
            
        return HealthCheck._instance
    def __init__(self):
        self._servers = []
    def addServer(self):
        self._servers.append("Server 1")
        self._servers.append("Server 2")
        self._servers.append("Server 3")
        self._servers.append("Server 4")
    def changeServer(self):
        self._servers.pop()
        self._servers.append("Server 5")


hc1 = HealthCheck()
hc2 = HealthCheck()

hc1.addServer()
print("Schedule health check for servers (1)..")
for i in range(4):
    print("Checking", hc1._servers[i])
    
hc2.changeServer()
print("Schedule health check for servers (2)..")

for i in range(4):
    print("Checing ", hc2._servers[i])

Schedule health check for servers (1)..
Checking Server 1
Checking Server 2
Checking Server 3
Checking Server 4
Schedule health check for servers (2)..
Checing  Server 1
Checing  Server 2
Checing  Server 3
Checing  Server 5



## 싱글톤 패턴의 단점

효율적이지만 단점도 있음. 싱글톤의 단일 전역 객체는 다음과 같은 문제점이 있음

* 전역 변수의 값이 실수로 변경된 것을 모르고 애플리케이션에서 사용될 수있음

* 같은 객체에 대한 여러 참조자가 생길 수 있음. 싱글톤은 한 개의 객체만을 만들기 때문에 같은 객체에 대한 여러 개의 참조자가 생김.

* 전역 변수에 종속적인 모든 클래스 간 상호관계가 복잡해짐. 전역 변수 수정이 의도치 않게 다른 변수에도 영향을 줄 수 있음.

> 다음의 내용을 반드시 기억하라.
 * 어플리케이션을 개발할 때 스레드 풀 과 캐시, 대화 상자, 레지스트리 설정 등 한 개의 객체만 필요한 경우가 많다. 이런 상황에서 여러 개의 객체를 생성하는 것은 리소스 낭비다. 따라서 싱글톤 패턴이 적합하다. 
 * 싱글톤은 글로벌 액세스 지점을 제공하는, 단점이 거의 없는 검증된 패턴인다.
 * 싱글톤 패턴의 단점은 전역 변수가 의도치 않게 다른 클래스에게 영향을 줄 수 있으며 리소스를 많이 사용하는 구조가 될 수 있따. 



다음 장 팩토리 패턴